<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/0127_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
tokens = ["딸기", "바나나", "사과", "딸기", "파인애플"]
# nodes =  list(set(tokens))
vocab = ["바나나", "사과", "파인애플", "딸기"]

vocab2id = { vocab[i] : i for i in range(len(vocab))}
id2vocab = { i : vocab[i] for i in range(len(vocab))}

print(vocab2id)
print(id2vocab)

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}
{0: '바나나', 1: '사과', 2: '파인애플', 3: '딸기'}


In [ ]:
import numpy as np

vocab_len = len(vocab)
weight_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)
scores = np.ones(vocab_len, dtype=np.float32)

window_size = 2

covered_coocurrences = []

for window_start in range(0, len(tokens)-window_size + 1):
    window = tokens[window_start : window_start + window_size]
    for i in range(window_size - 1):
        for j in range(i+1, window_size):
            
            if window[i] in vocab and window[j] in vocab:
                index_of_i = window_start + i
                index_of_j = window_start + j

                if (index_of_i, index_of_j) not in covered_coocurrences:
                    weight_edge[vocab2id[window[i]]][vocab2id[window[j]]] = 1
                    weight_edge[vocab2id[window[j]]][vocab2id[window[i]]] = 1
                    covered_coocurrences.append((index_of_i, index_of_j))

print(weight_edge)

for i in range(vocab_len):
    row_sum =  weight_edge[i].sum()
    weight_edge[i] = weight_edge[i] / row_sum if row_sum > 0 else 0
    
                
                
weight_edge          

[[0. 1. 0. 1.]
 [1. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 1. 1. 0.]]


array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        ]], dtype=float32)

In [ ]:
weight_edge[:, 0].shape

(4,)

In [ ]:
scores.shape

(4,)

In [ ]:
MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001

for it in range(MAX_ITERATIONS):
    prev_score = np.copy(scores)
    
    for i in range(vocab_len):
        
        summation = 0
        
        for j in range(vocab_len):            
            summation += weight_edge[j][i] * prev_score[j]
            
        scores[i] = (1 - d) + d * summation
        
    if np.sum(np.fabs(prev_score - scores)) <= threshold:
        break
        
scores

array([0.9837155 , 0.9837155 , 0.56564105, 1.4669281 ], dtype=float32)

In [ ]:
np.argsort(scores)

array([2, 0, 1, 3], dtype=int64)

In [ ]:
sorted_index = np.flip(np.argsort(scores), 0)
sorted_index

array([3, 1, 0, 2], dtype=int64)

In [ ]:
n = 3
for i in range(n):
    print("{} : {} ".format(id2vocab[sorted_index[i]], scores[sorted_index[i]]))

딸기 : 1.4669281244277954 
사과 : 0.9837154746055603 
바나나 : 0.9837154746055603 


In [ ]:
MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001

def get_scores(weight_edge, vocab_len):
    scores = np.ones(vocab_len, dtype=np.float32)
    
    for it in range(MAX_ITERATIONS):
        prev_score = np.copy(scores)

        for i in range(vocab_len):
            summation = (weight_edge[:, i] * prev_score).sum()
            scores[i] = (1 - d) + d * summation

        if np.sum(np.fabs(prev_score - scores)) <= threshold:
            break

    return scores

In [ ]:
def get_weight_edge(vocab, tokens, window_size = 2):
    vocab_len = len(vocab)
    weight_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)
    covered_coocurrences = []

    for window_start in range(0, len(tokens)-window_size + 1):
        window = tokens[window_start : window_start + window_size]
        for i in range(window_size - 1):
            for j in range(i+1, window_size):

                if window[i] in vocab and window[j] in vocab:
                    index_of_i = window_start + i
                    index_of_j = window_start + j

                    if (index_of_i, index_of_j) not in covered_coocurrences:
                        weight_edge[vocab2id[window[i]]][vocab2id[window[j]]] = 1
                        weight_edge[vocab2id[window[j]]][vocab2id[window[i]]] = 1
                        covered_coocurrences.append((index_of_i, index_of_j))

    for i in range(vocab_len):
        row_sum =  weight_edge[i].sum()
        weight_edge[i] = weight_edge[i] / row_sum if row_sum > 0 else 0

    return weight_edge          

In [ ]:
def preprocess(news_article, node_pos= ["NNG", "NNP"]):
    
    vocab = [token[0] for token in mecab.pos(news_article) if token[1] in node_pos]
    tokens = [token[0] for token in mecab.pos(news_article)]
    
    vocab = list(set(vocab))
             
    vocab2id = { vocab[i] : i for i in range(len(vocab))}
    id2vocab = { i : vocab[i] for i in range(len(vocab))}
    
    vocab_len = len(vocab)
    
    return vocab, tokens, vocab2id, id2vocab, vocab_len

In [ ]:
def report_result(scores, id2vocab):
    sorted_index = np.flip(np.argsort(scores), 0)
    
    print("핵심키워드")
    for i in range(top_n):        
        print("{} : {} ".format(id2vocab[sorted_index[i]], scores[sorted_index[i]]))
    print("")

In [ ]:
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()


def get_news(url):
    headers = {"user-agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article = html.select_one("#articleBodyContents").text
    return article

urls = ["https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728"]

docs = []
top_n = 5

for url in urls:
    news_article = get_news(url)
    
    vocab, tokens, vocab2id, id2vocab, vocab_len = preprocess(news_article)
    
    weight_edge = get_weight_edge(vocab, tokens, window_size=2)
    scores = get_scores(weight_edge, vocab_len)
    
    report_result(scores, id2vocab)
    

    
    

핵심키워드
데이터 : 7.4343390464782715 
활용 : 2.795384645462036 
정보 : 2.7120308876037598 
플랫 : 2.188546657562256 
경제 : 2.1860430240631104 

핵심키워드
대회 : 1.6575965881347656 
한전 : 1.6287670135498047 
서비스 : 1.5968517065048218 
선정 : 1.4593236446380615 
모델 : 1.4593236446380615 

핵심키워드
주행 : 4.062406063079834 
데이터 : 3.5409367084503174 
자율 : 2.3876307010650635 
산업 : 1.9965639114379883 
뉴스 : 1.9186471700668335 

핵심키워드
인구 : 3.6806445121765137 
지역 : 2.842867136001587 
간 : 2.663395404815674 
이동 : 2.6485180854797363 
데이터 : 2.024946689605713 

핵심키워드
데이터 : 4.3792619705200195 
금융 : 4.025979995727539 
중계 : 3.5793404579162598 
인프라 : 2.860461711883545 
개발 : 2.756382703781128 

